<p><img style="padding: 0 15px; float: left;" src="Images/FTLogo300.jpg" alt="FT Crusader Logo" Width='140' Height= '250'/></p> 
<p> <h2> Frogtown DataLoad Workbook, 05/06/18</h2> <a name="tc"></a>
<p> <h3> By Frogtown Crusader (Abu Nayeem)</h3>

This is just clean coded process going over the uploading steps assuming all the preliminary steps after decoder are made


## Table of contents <a name="content"></a>
* [Data Setup](#setup)
* [Create Variables](#create)
* [Intersection Table: Preparation](#intersection)
* [Address Table: Preparation](#address)
* [Preliminary Data Cleaning](#pre)


### Data Setup <a name="setup"></a>

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline 
from IPython.display import HTML
from IPython.display import display
import requests # library to handle requests

#Load Data
#df_crime = pd.read_csv('Datasets/Crime_Incident_Report_-_Dataset.csv')

#rename columns
#cols= ['Case','Date','Time','Code','IncType','Incident','Grid','NNum','Neighborhood','Block','CallDispCode','CallDisposition', 'Count']
#df_crime.columns= cols

#selection for Frogtown and nearby area
#df=df_crime.query('Grid in [66,67, 68, 86, 87,88,89, 90, 91, 92,106,107,108,109,110]')

#Socarata upload Method
from sodapy import Socrata

#New Upload Method Get Information from Socrata API
client = Socrata("information.stpaul.gov", None)

#Easier to bulk upload
results = client.get("gppb-g9cg", limit=1000000)
df = pd.DataFrame.from_records(results)
# Find Max Date Value
results = client.get("gppb-g9cg", limit=1)
r_max = pd.DataFrame.from_records(results)

#rename columns [Note the order of Columns have changed]
cols= ['Block','CallDispCode','CallDisposition','Case','Code', 'Count','Date','Grid','Incident','IncType','Neighborhood','NNum','Time']
df.columns= cols
df=df.dropna()
df = df.astype({"Case": int, "Code": int, "Grid":float, "NNum":int,"Count":int})
#select respective Grids of interest
df=df.query("Grid in (66,67, 68, 86, 87,88,89, 90, 91, 92,106,107,108,109,110)")

df.head(4)

,Block,CallDispCode,CallDisposition,Case,Code,Count,Date,Grid,Incident,IncType,Neighborhood,NNum,Time
7,127X SEMINARY AV,A,Advised,21023388,9954,1,2021-02-03T23:32:53.000,66.0,Proactive Police Visit,Proactive Police Visit,11 - Hamline/Midway,11,2021-02-03T23:32:53.000
11,30X FULLER AV,A,Advised,21023386,700,1,2021-02-03T23:28:28.000,110.0,Auto Theft,Motor Vehicle Theft,8 - Summit/University,8,2021-02-03T23:28:28.000
14,70X IGLEHART AV,A,Advised,21023377,9954,1,2021-02-03T23:23:37.000,108.0,Proactive Police Visit,Proactive Police Visit,8 - Summit/University,8,2021-02-03T23:23:37.000
15,26X UNIVERSITY AV E,A,Advised,21023376,9954,1,2021-02-03T23:22:37.000,92.0,Proactive Police Visit,Proactive Police Visit,5 - Payne/Phalen,5,2021-02-03T23:22:37.000


### Create New Variables <a name="create"></a>

In [2]:
#Add Time Variables
df= df[df.Case != 18254093] #messed up time variable

#Convert Date to Datetime!
from datetime import datetime

df['DateTime']= pd.to_datetime(df['Date']) # Create new column called DateTime
df['Year']= df['DateTime'].dt.year #create year column
df['DayofWeek']=df['DateTime'].dt.dayofweek #create day of the week column where default 0=Monday
df['Weekend'] = df['DayofWeek'].apply(lambda x: 1 if (x>4)  else 0) #Create a weekend category
df['Month'] = df['DateTime'].dt.month # Create Month Category
df['Day'] = df['DateTime'].dt.day #Create Day of the Current month
df['DayYear'] = df['DateTime'].dt.dayofyear  #Create Day of the year (0-365)
df['Day_Max'] = df.iloc[0,-1] #selects uptodate day; NOTE: the data is sorted chronologically

#Hour Data
df['TimeHour']= pd.to_datetime(df['Time'])
df['Hour'] = df['TimeHour'].dt.hour.astype(int) #Create Hour Colum
df['LateNight'] = df['Hour'].apply(lambda x: 1 if (x>21 or x<5)  else 0) #Latenight designation from 10Pm to 6PM

#Creating the intersection Column. Note: the Block column has the address information
df.Block = df.Block.astype(str) #first change the type to string
df['Block']= df['Block'].str.lower() #lowercase string to create uniformity

#While scanning the data I noticed that all intersections had "&" 
df['Intersection'] = df['Block'].apply(lambda x: 1 if '&' in x else 0) #intersection

df.head(5)

,Block,CallDispCode,CallDisposition,Case,Code,Count,Date,Grid,Incident,IncType,...,DayofWeek,Weekend,Month,Day,DayYear,Day_Max,TimeHour,Hour,LateNight,Intersection
7,127x seminary av,A,Advised,21023388,9954,1,2021-02-03T23:32:53.000,66.0,Proactive Police Visit,Proactive Police Visit,...,2,0,2,3,34,34,2021-02-03 23:32:53,23,1,0
11,30x fuller av,A,Advised,21023386,700,1,2021-02-03T23:28:28.000,110.0,Auto Theft,Motor Vehicle Theft,...,2,0,2,3,34,34,2021-02-03 23:28:28,23,1,0
14,70x iglehart av,A,Advised,21023377,9954,1,2021-02-03T23:23:37.000,108.0,Proactive Police Visit,Proactive Police Visit,...,2,0,2,3,34,34,2021-02-03 23:23:37,23,1,0
15,26x university av e,A,Advised,21023376,9954,1,2021-02-03T23:22:37.000,92.0,Proactive Police Visit,Proactive Police Visit,...,2,0,2,3,34,34,2021-02-03 23:22:37,23,1,0
16,dale st n & lafond,A,Advised,21023374,9954,1,2021-02-03T23:21:31.000,89.0,Proactive Police Visit,Proactive Police Visit,...,2,0,2,3,34,34,2021-02-03 23:21:31,23,1,1


### Prepare Intersection table <a name="intersection"></a>

In [3]:
#Load clean intersections key
df_Intersection_key = pd.read_csv('Datasets/Intersection_key_clean.csv')

# Create a new dateframe specifying only intersections from primary dataset
dfI=df.query('Intersection ==1')
print('The intersection table dimension are ' + str(dfI.shape))
#print(dfI.Block.head(10))


#Split the strings
new=dfI['Block'].str.split("& ", n = 1, expand = True) 
dfI['Inter2']= new[1]
new=dfI['Block'].str.split(" ", n = 1, expand = True) #Note the code specifies the first time a space occured
dfI['Inter1']=new[0]

#Create the IndexKey; recall we prepared the IntersectionKey where it considers any order
dfI['IndexKey']= dfI['Inter1']+ '_' + dfI['Inter2']
dfI.reset_index()
dfI=pd.merge(dfI, df_Intersection_key, on='IndexKey', how='left')



The intersection table dimension are (11429, 25)


In [4]:
#Drop rows with missing coordinates
dfI=dfI[dfI['Coordinates'].notnull()]

# Separate Latitude and Longitude 
new=dfI['Coordinates'].str.split(",", n = 1, expand = True) 
# making seperate first name column from new data frame 
dfI['Latitude']= pd.to_numeric(new[0]) #pd.to_numeric convert it to float
dfI['Longitude']= pd.to_numeric(new[1])

#Renaming columns

dfI['Block']=dfI['OutputKey'] #for practical purposes it makes sense
Drop_col=['Inter2','Inter1', 'IndexKey', 'Coordinates', 'OutputKey']
dfI_Final=dfI.drop(Drop_col, axis=1,)
dfI_Final.head(5)


,Block,CallDispCode,CallDisposition,Case,Code,Count,Date,Grid,Incident,IncType,...,Month,Day,DayYear,Day_Max,TimeHour,Hour,LateNight,Intersection,Latitude,Longitude
0,dale_lafond,A,Advised,21023374,9954,1,2021-02-03T23:21:31.000,89.0,Proactive Police Visit,Proactive Police Visit,...,2,3,34,34,2021-02-03 23:21:31,23,1,1,44.960251,-93.126380
1,western_aurora,A,Advised,21023367,9954,1,2021-02-03T23:10:40.000,109.0,Proactive Police Visit,Proactive Police Visit,...,2,3,34,34,2021-02-03 23:10:40,23,1,1,44.954906,-93.116150
2,kent_lafond,A,Advised,21023362,9954,1,2021-02-03T23:03:57.000,89.0,Proactive Police Visit,Proactive Police Visit,...,2,3,34,34,2021-02-03 23:03:57,23,1,1,44.960254,-93.123811
3,dale_thomas,A,Advised,21023351,9954,1,2021-02-03T22:45:57.000,89.0,Proactive Police Visit,Proactive Police Visit,...,2,3,34,34,2021-02-03 22:45:57,22,1,1,44.959342,-93.126371
4,western_university,RR,Report Written,21022987,710,1,2021-02-03T11:30:00.000,90.0,Auto Theft,"Motor Vehicle Theft, Automobile",...,2,3,34,34,2021-02-03 11:30:00,11,0,1,44.955870,-93.116148


### Prepare Address Decoder Table <a name="address"></a>

It is so remarkably short

In [5]:
#Split Data
dfW=df.query('Intersection==0')

#Load Complete Decoder Key
df_C= pd.read_csv('Datasets/SemiKey.csv')
df_C= df_C[['Block','Latitude','Longitude']]

# Merge with the dataset and remove missing values
dC=pd.merge(dfW, df_C, on='Block', how='left')
dC=dC.fillna('Mi')
dC=dC.query('Latitude != "Mi"')

#Bringing the data back together 
fg= dfI.append(dC, ignore_index=True)

### Preliminary Pre-Cleaning steps <a name="clean"></a>

It is good to do the data pre-cleaning steps here to reduce clutter on a working notebook. A few edits include renaming some values, clustering crimes together and creating some dummies. It is saved in a csv, which is used for execution

In [6]:
#Few Quick Edits
fg.CallDisposition.loc[(fg['CallDisposition'] == 'G - Gone on Arrival')] = 'Gone on Arrival'
fg.CallDisposition.loc[(fg['CallDisposition'] == 'A - Advised')] = 'Advised'
fg.CallDisposition.loc[(fg['CallDisposition'] == 'RR - Report Written')] = 'Report Written'
fg.Incident.loc[(fg['Incident'] == 'Simple Asasult Dom.')] = 'Simple Assault Dom.'
fg.Incident.loc[(fg['Incident'] == 'Graffiti')] = 'Vandalism'
fg.Incident.loc[fg["Incident"].isin([ "Rape","Agg. Assault",'Homicide'])]= 'Violent'
fg.Incident.loc[fg["Incident"].isin(["Simple Assault Dom.","Agg. Assault Dom."])]= 'Domestic Assault'


#[fg["Incident"].isin(["Simple Assault Dom.", "Rape"])


#Create a dummy for each crime category
fg= pd.concat([fg,pd.get_dummies(fg['Incident'])], axis=1)
fg= pd.concat([fg,pd.get_dummies(fg['CallDisposition'])], axis=1)

fg.to_csv('Datasets/FGCrime_Final.csv', encoding='utf-8', index=False)

In [7]:
fg.query('Year==2020')

,Block,CallDispCode,CallDisposition,Case,Code,Coordinates,Count,Date,DateTime,Day,...,Vandalism,Violent,10 - Como,11 - Hamline/Midway,13 - Union Park,7 - Thomas/Dale(Frogtown),8 - Summit/University,Advised,Gone on Arrival,Report Written
248,arundel_edmund,A,Advised,20276624,9954,"44.958472, -93.118665",1,2020-12-31T21:16:46.000,2020-12-31 21:16:46,31,...,0,0,0,0,0,0,0,1,0,0
249,mackubin_blair,A,Advised,20276554,9954,"44.961158, -93.121254",1,2020-12-31T19:09:56.000,2020-12-31 19:09:56,31,...,0,0,0,0,0,0,0,1,0,0
250,mackubin_thomas,A,Advised,20276547,9954,"44.959365, -93.121234",1,2020-12-31T18:59:41.000,2020-12-31 18:59:41,31,...,0,0,0,0,0,0,0,1,0,0
251,arundel_thomas,A,Advised,20276481,9954,"44.959368, -93.118665",1,2020-12-31T17:17:02.000,2020-12-31 17:17:02,31,...,0,0,0,0,0,0,0,1,0,0
252,arundel_thomas,A,Advised,20276404,9954,"44.959368, -93.118665",1,2020-12-31T15:35:36.000,2020-12-31 15:35:36,31,...,0,0,0,0,0,0,0,1,0,0
253,linden_arch,A,Advised,20276350,9954,"44.960767, -93.095135",1,2020-12-31T14:20:42.000,2020-12-31 14:20:42,31,...,0,0,0,0,0,0,0,1,0,0
254,syndicate_seminary,A,Advised,20276068,9954,"44.964755, -93.154270",1,2020-12-31T01:46:44.000,2020-12-31 01:46:44,31,...,0,0,0,0,0,0,0,1,0,0
255,grotto_thomas,A,Advised,20276004,9954,"44.959356, -93.131431",1,2020-12-30T22:59:50.000,2020-12-30 22:59:50,30,...,0,0,0,0,0,0,0,1,0,0
256,griggs_seminary,A,Advised,20275635,9954,"44.964777, -93.151718",1,2020-12-30T13:18:40.000,2020-12-30 13:18:40,30,...,0,0,0,0,0,0,0,1,0,0
257,kent_blair,A,Advised,20275446,9954,"44.961153, -93.123816",1,2020-12-30T05:29:55.000,2020-12-30 05:29:55,30,...,0,0,0,0,0,0,0,1,0,0


In [7]:
Ward1.query('ElectionDescription in ("11/08/2016 - STATE GENERAL", "08/09/2016 - STATE PRIMARY","11/07/2017 - MUNICIPAL GENERAL","11/07/2017 - SCHOOL DISTRICT GENERAL"  ,"11/08/2016 - STATE GENERAL","08/09/2016 - STATE PRIMARY","11/03/2015 - MUNICIPAL GENERAL","11/03/2015 - SCHOOL DISTRICT GENERAL","11/08/2011 - MUNICIPAL GENERAL","11/06/2018 - STATE GENERAL","08/14/2018 - STATE PRIMARY")' 

SyntaxError: unexpected EOF while parsing (<ipython-input-7-b3cfb0eeffb8>, line 1)